# Pointcloud shifthing


After estimating the pose of the ArUco tag and transforming the coordinates from the CAD model's coordinate system (i.e., the demonstrator) to the scan's coordinate system (i.e., the point cloud), I noticed an issue with the depth estimation of the MechMind scanner. This observation stems from the fact that the point cloud and the CAD model (i.e., the demonstrator) are not perfectly (or nearly perfectly) aligned. To address this, I decided to find the nearest point in the point cloud to the corresponding ArUco tag on the CAD model and shift the point cloud by the distance between them.

![Txt](images/one.png)

First, I define the depth directions of the ArUco tags e.g. ArUco tags 5 and 8 have their depth directions along the x-axis.

In [6]:
import open3d as o3d
import detect
import numpy as np
import glob
import cv2
import json

# ArUco size in meter:  0.105
# 0,0,0 means the back of the arUco points to the floor and the Kosy of the Demostrator 

depth_directions_of_arucos = {"x": [5,8],
                              "y": [1,2,3,4,6,7,10], 
                              "z": [9]}

Here, I defined some functions to load the CAD-model, the point cloud and the image taken from the camera (the camera makes a point cloud and an image simultaniously).

In [7]:
# CAD-model
def get_mesh(path): 
    mesh = o3d.io.read_triangle_mesh(path)
    mesh.compute_vertex_normals()
    return mesh

# Pointcloud
def get_path_to_pcd(path_to_scan):
    return glob.glob(path_to_scan + '*.ply')[0]

def get_pointcloud(path):
    point_cloud = o3d.io.read_point_cloud(path)
    return point_cloud

# Image
def get_path_to_image(path_to_scan):
    return glob.glob(path_to_scan + '*.png')[0]

This is the function that finds the nearest point from the point cloud to the center of the given ArUco (which is placed on the CAD-model).

In [8]:
def find_nearest_point_in_direction(pcd, reference_point, direction_vector):
    """
    Get the point that is nearest to the 'reference_point' in the pointcloud in the given direction

    Args: pcd = pointcloud from which we take the points
          reference point = the starting point from which we measure the direction
          direction_vector = the direction in which we will search the point
    Return: nearest point to the reference point   
    """

    # Normalize the direction vector (# in our case the vector is already normalized. this is for any case if we use some other direction thats not on the axis (e.g if the demostrator was hexagonal prism)
    direction_unit = direction_vector / np.linalg.norm(direction_vector)
    
    # Calculate the vector from the center of the Aruco to each point in the point cloud
    vectors_to_points = np.asarray(pcd.points) - reference_point
    
    # Project these vectors onto the direction unit vector
    projections = np.dot(vectors_to_points, direction_unit)
    
    # Calculate the distances from the projected points to the original points and find the eucliand distance to the point
    distances = np.linalg.norm(vectors_to_points - np.outer(projections, direction_unit), axis=1)
    
    # Find the index of the nearest point based on the minimum distance
    nearest_index = np.argmin(distances)
    
    # Get the nearest point
    nearest_point = pcd.points[nearest_index]
    
    return nearest_point

For each possible depth direction, I defined a function. Each function iterates through the detected ArUco markers, and if it finds an ArUco with the corresponding depth direction, it retrieves its coordinates from the CAD model. It then identifies the nearest point in the point cloud along the specified direction and calculates a translation vector. The function computes the mean translation vector from all the translation vectors of the detected ArUco markers in the given image and shifts the point cloud accordingly.

The specified regions of the code aren't a general solution. We know beforehand were we placed the ArUco tags and we know where the point cloud is.

In [9]:
def get_mean_translation_vector_y(pcd, ids):
    divide = 0

    mean_translation_vector_y = np.array([.0, .0, .0])
   
    for i in ids.flatten():
      if i in depth_directions_of_arucos["y"]:
         with open('tag_info.json', 'r') as f:
            data = json.load(f)
            translation = data["AruCo-Tag-Positionen (Tag-Mitte)"][str(i)]
            translation = np.array([translation])
            aruco_center = o3d.geometry.PointCloud()
            aruco_center.points = o3d.utility.Vector3dVector(translation)
            direction_vector = [.0, 1.0, .0]
            nearest_point = find_nearest_point_in_direction(pcd, translation, direction_vector)
            ####################
            # All arUco tags with depth direction y-axis are only on the left side of the demonstrator.
            # That's why this is enough.
            shift_distance = np.abs(translation[0][1] - nearest_point[1])
            ####################
            translation_vector = np.array([.0, shift_distance, .0])
            print("ArUco Tag number: " + str(i) + " with depth direction in y axis")
            print("Translation vector of the give ArUco Tag",translation_vector)
            mean_translation_vector_y += translation_vector
            divide+=1

    if divide != 0:
       mean_translation_vector_y /= divide

    return mean_translation_vector_y   

def get_mean_translation_vector_x(pcd,ids):  
    divide = 0  
    
    mean_translation_vector_x = np.array([.0, .0, .0])

    for i in ids.flatten():
      if i in depth_directions_of_arucos["x"]:
         with open('tag_info.json', 'r') as f:
            data = json.load(f)
            translation = data["AruCo-Tag-Positionen (Tag-Mitte)"][str(i)]
            translation = np.array([translation])
            aruco_center = o3d.geometry.PointCloud()
            aruco_center.points = o3d.utility.Vector3dVector(translation)
            direction_vector = [1.0, .0, .0]
            nearest_point = find_nearest_point_in_direction(pcd, translation, direction_vector)
            ####################
            # We have arUco tag only on the front size of the demonstrator.
            # That's why this is enought.
            shift_distance = translation[0][0] - nearest_point[0]
            ####################
            translation_vector = np.array([shift_distance, .0, .0])
            print("ArUco Tag number: " + str(i) + " with depth direction in x axis")
            print("Translation vector of the give ArUco Tag",translation_vector)
            mean_translation_vector_x += translation_vector
            divide+=1

    if divide != 0:
       mean_translation_vector_x /= divide

    return mean_translation_vector_x   

def get_mean_translation_vector_z(pcd,ids):  
    divide = 0  
    
    mean_translation_vector_z = np.array([.0, .0, .0])

    for i in ids.flatten():
      if i in depth_directions_of_arucos["z"]:
         with open('tag_info.json', 'r') as f:
            data = json.load(f)
            translation = data["AruCo-Tag-Positionen (Tag-Mitte)"][str(i)]
            translation = np.array([translation])
            aruco_center = o3d.geometry.PointCloud()
            aruco_center.points = o3d.utility.Vector3dVector(translation)
            direction_vector = [.0, .0, 1.0]
            nearest_point = find_nearest_point_in_direction(pcd, translation, direction_vector)
            ####################
            # We have only one arUco at the top of the demonstrator.
            # That's why this is enough.
            shift_distance = translation[0][2] - nearest_point[2]
            ####################
            translation_vector = np.array([.0, .0, shift_distance])
            print("ArUco Tag number: " + str(i) + " with depth direction in z axis")
            print("Translation vector of the give ArUco Tag", translation_vector)
            mean_translation_vector_z += translation_vector
            divide+=1

    if divide != 0:
       mean_translation_vector_z /= divide

    return mean_translation_vector_z    

An example demonstrating the code's usage and showcasing the obtained results

In [10]:
path_to_mesh = "data/demonstrator_model/scaled_demonstrator.stl"
path_to_pcd = "data/scans_roughly_aligned/10_Mean.ply"
path_to_image = "data/Mech_Eye/2024-01-19/10/"

# load the CAD-file
mesh = get_mesh(path_to_mesh)
# load the point cloud
pcd = get_pointcloud(path_to_pcd)
  
image = cv2.imread(get_path_to_image(path_to_image))
corners, ids, _ = detect.detect_all_markers(image, "DICT_4X4_100")
print("Detected ArUcos in the image:", ids.flatten())
    
mean_translation_vector = get_mean_translation_vector_x(pcd, ids) + get_mean_translation_vector_y(pcd, ids) + get_mean_translation_vector_z(pcd,ids)
print("Mean translation vector:", mean_translation_vector)
pcd = pcd.translate(mean_translation_vector)

pcd_original = get_pointcloud(path_to_pcd)   
o3d.visualization.draw_geometries([mesh, pcd_original], window_name="Demostrator and Pointcloud Mean", width=800, height=600) 
o3d.visualization.draw_geometries([mesh, pcd], window_name="Demostrator and Pointcloud", width=800, height=600)     

Detected ArUcos in the image: [5 8 9]
ArUco Tag number: 5 with depth direction in x axis
Translation vector of the give ArUco Tag [-0.03372341  0.          0.        ]
ArUco Tag number: 8 with depth direction in x axis
Translation vector of the give ArUco Tag [-0.02429521  0.          0.        ]
ArUco Tag number: 9 with depth direction in z axis
Translation vector of the give ArUco Tag [ 0.          0.         -0.00407399]
Mean translation vector: [-0.02900931  0.         -0.00407399]


![Text](images/two.png)

Although the achieved results are good, another issue was identified. Due to the depth estimation problem of the scanner, not only is shifting required, but also rotation, as the left side of the demonstrator is closer to the camera than the right side.